In [1]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd


models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/fastspeech2-en-ljspeech",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)


2022-02-10 13:38:00 | INFO | fairseq.tasks.speech_to_text | dictionary size (vocab.txt): 75


In [9]:
model = models[0]
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)
generator = task.build_generator(models, cfg)

text = "Hello, this is a test run."

model.to("cuda:0")

sample = TTSHubInterface.get_model_input(task, text)
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

ipd.Audio(wav, rate=rate)

2022-02-10 13:54:46 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from C:/Users/runze/.cache/fairseq/facebook--fastspeech2-en-ljspeech.main.a3e3e5e2e62bb7ca7514b11aa469e9c5b01a20bf/hifigan.bin


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)